## Importing directories

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import openpyxl

In [6]:
## Read the files

df_entriesgender= pd.read_excel(r'C:\Users\manis\OneDrive\Desktop\Jupyter notebook codes\Jupyter codes\SQL project\EntriesGender.xlsx')
df_teams= pd.read_excel(r'C:\Users\manis\OneDrive\Desktop\Jupyter notebook codes\Jupyter codes\SQL project\Teams.xlsx')
df_athletes= pd.read_excel(r'C:\Users\manis\OneDrive\Desktop\Jupyter notebook codes\Jupyter codes\SQL project\Athletes.xlsx')
df_coaches= pd.read_excel(r'C:\Users\manis\OneDrive\Desktop\Jupyter notebook codes\Jupyter codes\SQL project\Coaches.xlsx')
df_medals= pd.read_excel(r'C:\Users\manis\OneDrive\Desktop\Jupyter notebook codes\Jupyter codes\SQL project\Medals.xlsx')

D:\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
## Read all the columns of the files

print('The columns of the gender table are:',df_entriesgender.columns.to_list())
print('\n')
print('The columns of the teams table are:',df_teams.columns.to_list())
print('\n')
print('The columns of the athletes table are:',df_athletes.columns.to_list())
print('\n')
print('The columns of the coaches table are:',df_coaches.columns.to_list())
print('\n')
print('The columns of the medals table are:',df_medals.columns.to_list())
print('\n')

The columns of the gender table are: ['Discipline', 'Female', 'Male', 'Total']


The columns of the teams table are: ['Name', 'Discipline', 'NOC', 'Event']


The columns of the athletes table are: ['Name', 'NOC', 'Discipline']


The columns of the coaches table are: ['Name', 'NOC', 'Discipline', 'Event']


The columns of the medals table are: ['Rank', 'Team/NOC', 'Gold', 'Silver', 'Bronze', 'Total', 'Rank by Total']




In [8]:
## Renaming a few columns

df_entriesgender_SQL = df_entriesgender # no change required

df_teams_SQL = df_teams # no change required

df_athletes_SQL = df_athletes # no change required

df_coaches_SQL = df_coaches # no change required

df_medals_SQL = df_medals.rename(columns = {'Team/NOC': 'Teamnoc',
                              }, inplace = False)

In [17]:
# import sqlalchemy and create a sqlite engine
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# export the dataframe as a table 'playstore' to the sqlite engine
df_entriesgender_SQL.to_sql("EntriesGender", con =engine)
df_teams_SQL.to_sql("Teams", con =engine)
df_athletes_SQL.to_sql("Athletes", con =engine)
df_coaches_SQL.to_sql("Coaches", con =engine)
df_medals_SQL.to_sql("Medals", con =engine)

93

In [19]:
##  Test the Query

sql='''

Select * from entriesgender

''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,Discipline,Female,Male,Total
0,0,3x3 Basketball,32,32,64
1,1,Archery,64,64,128
2,2,Artistic Gymnastics,98,98,196
3,3,Artistic Swimming,105,0,105
4,4,Athletics,969,1072,2041


In [20]:
##  Query 1: - Avg count of participants across all disciplines

sql='''

Select
discipline,
total as Participants
from entriesgender
group by discipline
order by total desc
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Discipline,Participants
0,Athletics,2041
1,Swimming,779
2,Football,608
3,Rowing,522
4,Judo,393


In [21]:
##  Query 2: - Most Medals won by each country, top 3 ranks

sql='''

select *
from medals
order by total desc
limit 3

''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,Rank,Teamnoc,Gold,Silver,Bronze,Total,Rank by Total
0,0,1,United States of America,39,41,33,113,1
1,1,2,People's Republic of China,38,32,18,88,2
2,4,5,ROC,20,28,23,71,3


In [22]:
##  Query 3: - Most Bronze,silver and gold

sql='''

select 'Most Gold  :- ' || teamnoc as Country_With_Most_Medals,max(gold) as Medals from medals
union all
select 'Most Silver :- ' || teamnoc as Country_With_Most_Medals,max(silver) as Medals from medals
union all
select 'Most Bronze  :- ' || teamnoc as Country_With_Most_Medals,max(bronze) as Medals from medals


''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Country_With_Most_Medals,Medals
0,Most Gold :- United States of America,39
1,Most Silver :- United States of America,41
2,Most Bronze :- United States of America,33


In [23]:
##  Query 4: - Particpants at across countries

sql='''
select * from (
select count(name) as Count_of_Players,NOC
from athletes
group by noc) a
order by a.count_of_players desc


''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Count_of_Players,NOC
0,615,United States of America
1,586,Japan
2,470,Australia
3,401,People's Republic of China
4,400,Germany


In [24]:
##  Query 5: - Numbers Table, count of various events
sql='''
select 'Count of Events :- '|| count(distinct discipline) as Number from athletes
union all
select 'Count of Countries :- '|| count(distinct noc) as Number from athletes
union all
select 'Total Female Athletes :- '|| sum(female) as Number from EntriesGender
union all
select 'Total Male Athletes :- '|| sum(male) as Number from EntriesGender
union all
select 'Total Athletes :- '|| sum(total) as Number from EntriesGender
union all
select 'Total Gold :- '|| sum(gold) as Number from medals
union all
select 'Total Silver :- '|| sum(silver) as Number from medals
union all
select 'Total Bronze :- '|| sum(bronze) as Number from medals
union all
select 'Total Medals :- '|| sum(total) as Number from medals


''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head(10)

,Number
0,Count of Events :- 46
1,Count of Countries :- 206
2,Total Female Athletes :- 5432
3,Total Male Athletes :- 5884
4,Total Athletes :- 11316
5,Total Gold :- 340
6,Total Silver :- 338
7,Total Bronze :- 402
8,Total Medals :- 1080


In [25]:
##  Query 6: - Coaches produced by the countries

sql='''
select * from (
select count(name) as Count_of_Coaches,noc as NOC from coaches
group by noc) a
order by a.Count_of_Coaches desc

''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Count_of_Coaches,NOC
0,35,Japan
1,28,Spain
2,28,United States of America
3,22,Australia
4,16,Canada


In [26]:
##  Query 7: - Coaches vs Player Ratio

sql='''


select
coach_table.NOC,
Count_of_Players,
Count_of_Coaches,
round(Count_of_Players/Count_of_Coaches,2) as Player_Coach_Ratio

from 
(select * from (
select count(name) as Count_of_Players,NOC
from athletes
group by noc) a
order by a.count_of_players desc) player_table

join (
select * from (
select count(name) as Count_of_Coaches,noc as NOC from coaches
group by noc) a
order by a.Count_of_Coaches desc) coach_table

on player_table.noc=coach_table.noc

''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,NOC,Count_of_Players,Count_of_Coaches,Player_Coach_Ratio
0,Angola,20,1,20.0
1,Argentina,180,11,16.0
2,Australia,470,22,21.0
3,Austria,72,3,24.0
4,Bahrain,31,1,31.0


In [27]:
##  Query 8: - Country wise Performance Table

sql='''

select 
count(c.name) as Coach_Team,
count_of_players as Player_Team,
c.noc,
m.total as Total_Medals_Won

from coaches c
join (select 
teamnoc as country,
gold,
silver,
bronze,
total
from medals) m
on c.noc=m.country 
join (select * from (
select count(name) as Count_of_Players,NOC
from athletes
group by noc) a
order by a.count_of_players desc) p
on c.noc=p.noc
group by c.noc
order by m.total desc
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Coach_Team,Player_Team,NOC,Total_Medals_Won
0,28,615,United States of America,113
1,12,401,People's Republic of China,88
2,12,318,ROC,71
3,7,366,Great Britain,65
4,35,586,Japan,58


In [28]:
##  Query 9: - Sports with Highes female participation

sql='''
select * from (
select
discipline as Discipline,
sum(female) as Count_of_Female,
sum(total) as Count_of_Total,
100*(round(round(female,4)/round(total,4),4))as Female_Participation
from entriesgender
group by discipline)
order by Female_Participation desc
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Discipline,Count_of_Female,Count_of_Total,Female_Participation
0,Artistic Swimming,105,105,100.00
1,Rhythmic Gymnastics,96,96,100.00
2,Cycling BMX Freestyle,10,19,52.63
3,Diving,72,143,50.35
4,3x3 Basketball,32,64,50.00


In [29]:
##  Query 10: - Sports with Highes male participation

sql='''
select * from (
select
discipline as Discipline,
sum(Male) as Count_of_Male,
sum(total) as Count_of_Total,
100*(round(round(male,4)/round(total,4),4))as Male_Participation
from entriesgender
group by discipline)
order by Male_Participation desc
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Discipline,Count_of_Male,Count_of_Total,Male_Participation
0,Wrestling,193,289,66.78
1,Cycling Road,131,201,65.17
2,Boxing,187,289,64.71
3,Equestrian,125,198,63.13
4,Baseball/Softball,144,234,61.54


In [30]:
##  Query 11: -  Player vs coach List

sql='''

select
a.name as Player_Name,
coach.coach_name as Coach_Name,
a.noc,
a.discipline as discipline

from athletes a
join (
select 
c.name as coach_name,
c.noc as country,
c.discipline as discipline from coaches c) coach
on coach.country=a.noc and coach.discipline=a.discipline
group by
a.noc,
coach.coach_name
order by discipline asc
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Player_Name,Coach_Name,NOC,discipline
0,BUCKLE Carolyn,MONTICO Loredana,Australia,Artistic Swimming
1,ALEXANDRI Anna-Maria,BRUN Aristide,Austria,Artistic Swimming
2,ALEXANDRI Anna-Maria,MLADENOVA Albena,Austria,Artistic Swimming
3,ALEXANDRI Anna-Maria,SHAHNAZI Shahbaz,Austria,Artistic Swimming
4,KHANDOSHKA Vasilina,SVETLICHNAYA Aliona,Belarus,Artistic Swimming


In [31]:
##  Query 12: -  Teams vs Disciplines
sql='''

select * from (
select
Name,
count(distinct discipline) as Count_of_Discipline
from teams
group by name
) a
order by a.Count_of_Discipline desc
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Name,Count_of_Discipline
0,Japan,19
1,United States,17
2,China,16
3,Canada,15
4,France,15


In [32]:
## Import the dataset

df_continents= pd.read_excel(r'C:\Users\manis\OneDrive\Desktop\Jupyter notebook codes\Jupyter codes\SQL project\Continent List.xlsx')
df_continents.head()

,Continent,Country
0,Asia,Afghanistan
1,Europe,Albania
2,Africa,Algeria
3,Oceania,American Samoa
4,Europe,Andorra


In [34]:
## Rename the file for simplicity while writing SQL queries

df_copy= df_continents 
df_copy.to_excel('continents.xlsx',index=False)

In [35]:
## Read the Renamed File

df_continents  = pd.read_excel('continents.xlsx')
df_continents.head()

,Continent,Country
0,Asia,Afghanistan
1,Europe,Albania
2,Africa,Algeria
3,Oceania,American Samoa
4,Europe,Andorra


In [36]:
## Read all the columns of the files

print('The columns of the Continents table are:',df_continents.columns.to_list())

The columns of the Continents table are: ['Continent', 'Country']


In [37]:
df_continents_SQL=df_continents

In [38]:
df_continents_SQL.to_sql("continents", con = engine)

206

In [39]:
## Query 13:- Test the added contient table query

sql='''

select * from continents
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,Continent,Country
0,0,Asia,Afghanistan
1,1,Europe,Albania
2,2,Africa,Algeria
3,3,Oceania,American Samoa
4,4,Europe,Andorra


In [40]:
##  Query 14: -  Continent wise breakdown
sql='''

select 
distinct(name) as Country,
c.continent
from teams t
join continents c on t.name=c.country
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Country,Continent
0,Belgium,Europe
1,China,Asia
2,France,Europe
3,Italy,Europe
4,Japan,Asia


In [41]:
##  Query 15: -  Continent vs count of countries partcipated in the olympics


sql='''

select * 
from (
select 
continent, 
count(country) as No_of_Countries
from continents
group by continent) a

order by a.No_of_Countries desc
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head(10)

,continent,No_of_Countries
0,Africa,54
1,Asia,49
2,Europe,43
3,North America,28
4,Oceania,17
5,South America,13
6,None,2


In [42]:
##  Query 16: - Continents Performance Table

sql='''

select 
co.continent,
count(c.name) as Total_Coaches,
count_of_players as TOtal_Players,
m.total as Total_Medals_Won

from coaches c
join (select 
teamnoc as country,
gold,
silver,
bronze,
total
from medals) m
on c.noc=m.country 
join (select * from (
select count(name) as Count_of_Players,NOC
from athletes
group by noc) a
order by a.count_of_players desc) p
on c.noc=p.noc
join continents co on
c.noc=co.country
group by co.continent
order by m.total desc

''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Continent,Total_Coaches,TOtal_Players,Total_Medals_Won
0,Oceania,32,470,46
1,North America,33,368,24
2,Europe,132,72,7
3,Africa,39,133,6
4,South America,30,180,3
